In [21]:
import torch
from torchvision.transforms import transforms
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import pathlib 
from pathlib import Path

In [15]:
class CatDog(Dataset):

    def __init__(self, dataset_path: pathlib.Path):
        self.dataset_path = Path(dataset_path)
        self.image_paths = self.dataset_path.glob("*.jpg") 

        self.data = {"cats" : [], "dogs" : []}

        for ip in self.image_paths:
            if "cat" == ip.name[:3]:
                self.data["cats"].append(ip.read_bytes())
            elif "dog" == ip.name[:3]:
                self.data["dogs"].append(ip.read_bytes())

        # Legacy support
        self.data_list = list(self.image_paths)
        self.data_class = ["cat", "dog"]

        self.transform_func = transforms.Compose([
             transforms.ToPILImage(),
             transforms.Grayscale(),
             transforms.Resize((124, 124)),
             transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, index):
        img_path = self.data_list[index]
        img_label = torch.zeros(len(self.data_class))
        img_label[self.data_class.index(img_path.parent)] = 1.0 
        img_tensor = read_image(str(img_path))
        return self.transform_func(img_tensor), img_label
    

In [26]:
dataset_path = Path("../train/")
image_paths = dataset_path.glob("*.jpg") 
list(image_paths)

[WindowsPath('../train/cat.0.jpg'),
 WindowsPath('../train/cat.1.jpg'),
 WindowsPath('../train/cat.10.jpg'),
 WindowsPath('../train/cat.100.jpg'),
 WindowsPath('../train/cat.1000.jpg'),
 WindowsPath('../train/cat.10000.jpg'),
 WindowsPath('../train/cat.10001.jpg'),
 WindowsPath('../train/cat.10002.jpg'),
 WindowsPath('../train/cat.10003.jpg'),
 WindowsPath('../train/cat.10004.jpg'),
 WindowsPath('../train/cat.10005.jpg'),
 WindowsPath('../train/cat.10006.jpg'),
 WindowsPath('../train/cat.10007.jpg'),
 WindowsPath('../train/cat.10008.jpg'),
 WindowsPath('../train/cat.10009.jpg'),
 WindowsPath('../train/cat.1001.jpg'),
 WindowsPath('../train/cat.10010.jpg'),
 WindowsPath('../train/cat.10011.jpg'),
 WindowsPath('../train/cat.10012.jpg'),
 WindowsPath('../train/cat.10013.jpg'),
 WindowsPath('../train/cat.10014.jpg'),
 WindowsPath('../train/cat.10015.jpg'),
 WindowsPath('../train/cat.10016.jpg'),
 WindowsPath('../train/cat.10017.jpg'),
 WindowsPath('../train/cat.10018.jpg'),
 WindowsPath('.

In [23]:
for p in list(image_paths)[:10]:
    print(p.name)

cat.0.jpg
cat.1.jpg
cat.10.jpg
cat.100.jpg
cat.1000.jpg
cat.10000.jpg
cat.10001.jpg
cat.10002.jpg
cat.10003.jpg
cat.10004.jpg


In [18]:
# g = pathlib.Path(r'C:\Users\Joe\Desktop\python_yes\what_is_love\Latin')
# a = [x for x in g.glob('**/*') if x.is_file()]
# set([x.parent for x in a])
# #[a for a in g.iterdir()]

In [19]:
class CVModel(nn.Module):

    def __init__(self):
        super(CVModel, self).__init__()
        self.conv_1 = nn.Conv2d(1, 32, (3, 3))
        self.conv_2 = nn.Conv2d(32, 64, (3, 3))
        self.pool_1 = nn.MaxPool2d((2, 2))
        self.conv_3 = nn.Conv2d(64, 128, (3, 3))
        self.conv_4 = nn.Conv2d(128, 128, (3, 3))
        self.pool_2 = nn. MaxPool2d((2, 2))
        self.glob_pool = nn.MaxPool2d((28, 28))
        self.linear = nn.Linear(128, 2) #2 классa
        self.softmax = nn.Softmax()

    def forward(self, X):
        res = self.conv_1(X)
        res = self.conv_2(res)
        res = self.pool_1(res)
        res = self.conv_3(res)
        res = self.conv_4(res)
        res = self.pool_2(res)
        res = self.glob_pool(res) #(1, 1, 128)
        res = self.linear(res.view(-1, 128)) #Сокращение размерности до (128)
        return self.softmax(res)

In [20]:
path = pathlib.Path(r'../train/')
loss = nn.BCELoss()
dataset = CatDog(path)
model = CVModel()
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)
optimizer = torch.optim.RMSprop(params=model.parameters())

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [24]:
path = pathlib.Path(r'../train/')
loss = nn.BCELoss()
dataset = CatDog(path)
print(len(dataset))

0


In [ ]:
def train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset):
    correct = 0
    epoch_loss = 0.0 
    for i, data in enumerate(trainloader, 0):
        x, y = data
        optimizer.zero_grad()
        outputs = model(x)
        error = loss(outputs, y)
        error.backward()
        epoch_loss += outputs.shape[0] * error.item()
        optimizer.step()
        outputs = (outputs > 0.5).float()
        correct += (outputs == y).float().sum()

    print(f'TRAIN [{epoch + 1}] loss: {epoch_loss/ dataset.__len__():.3f}')

In [ ]:
for epoch in range(1):
    train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset)

c:\Users\Joe\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


TRAIN [1] loss: 3.283
